In [ ]:
qiime tools import --type 'SampleData[PairedEndSequencesWithQuality]' --input-path /Users/madibaltagulov/Documents/16s-Madi/TPT_diet-selected/TPT_manifest.tsv  --output-path paired-end-demux.qza --input-format PairedEndFastqManifestPhred33V2

Import sequencing files through the premade manifest file and respective. Create artifact file of quality scores for positions in our sequence for all reads. 

In [ ]:
qiime demux summarize --i-data paired-end-demux.qza --o-visualization paired-end-demux.qzv

Summarize and visualize the artifacts in a qzv file

In [ ]:
qiime dada2 denoise-paired --i-demultiplexed-seqs paired-end-demux.qza --p-trunc-len-f 150 --p-trunc-len-r 150 --p-trim-left-f 20 --p-trim-left-r 20 --o-representative-sequences dada2-rep-seqs.qza --o-table table.qza --o-denoising-stats stats-dada2.qza --p-n-threads 8


denoised the sequences we have, essentially trimming at bases that we decided have a low quality score (<25qc). 'trunc-len' is for 3' end bases, 'trim-left' is for 5' end bases. -f is for forward reads, -r for reverse reads.

In [ ]:
qiime metadata tabulate --m-input-file stats-dada2.qza --o-visualization dada2-stats.qzv

Makes a qzv file that makes the metadata and visualizing it in a table with denoised data

In [ ]:
qiime feature-table summarize --i-table table.qza --o-visualization table.qzv --m-sample-metadata-file /Users/madibaltagulov/Documents/16s-Madi/TPT_diet-selected/TPT_metadata.txt

This summarizes the feature table and pairs the metadata treatment groups with our sequences. A feature is a unique read (in V3-4 16s, that would be an Amplicon Sequence Variant (ASV)).

In [ ]:
qiime tools view 'insert file name'


allows to output any qzv file

In [ ]:
conda activate qiime2-2023.2

In [ ]:
cd 'file path'

sets up the directory to work in

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree --i-sequences dada2-rep-seqs.qza --o-alignment aligned-rep-seqs.qza --o-masked-alignment masked-aligned-rep-seqs.qza --o-tree unrooted-tree.qza --o-rooted-tree rooted-tree.qza


Creating a phylogeny tree

In [ ]:
qiime diversity alpha-rarefaction --i-table table.qza --i-phylogeny rooted-tree.qza --p-max-depth 27781 --m-metadata-file /Users/madibaltagulov/Documents/16s-Madi/TPT_diet-selected/TPT_metadata.txt --o-visualization alpha-rarefaction.qzv

Performing rarefaction (finding what minimum number of features/sample is appropriate to be representative of maximum amount of variation in each sample). This number should be sourced from the feature table qzv. Rarefaction was successful if every alpha diversity curve flatlines.

In [ ]:
qiime diversity core-metrics-phylogenetic --i-phylogeny rooted-tree.qza --i-table table.qza --p-sampling-depth 27781 --m-metadata-file /Users/madibaltagulov/Documents/16s-Madi/TPT_diet-selected/TPT_metadata.txt --output-dir core-metrics-results

Performing an array of alpha-diversity and beta-diversity tests on our reads, coupled with metadata categories and labels. Ouputs into a new folder.

In [ ]:
qiime diversity alpha-group-significance --i-alpha-diversity core-metrics-results/shannon_vector.qza --m-metadata-file /Users/madibaltagulov/Documents/16s-Madi/TPT_diet-selected/TPT_metadata.txt --o-visualization core-metrics-results/shannon-vector.qzv

An example of how to output an alpha-diversity vector visualization. Also performs a Kruskal-Wallis (on each pair of categories) test to see if the differences btw metadata categories are significant.

In [ ]:
qiime diversity beta-group-significance --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza --m-metadata-file /Users/madibaltagulov/Documents/16s-Madi/TPT_diet-selected/TPT_metadata.txt --m-metadata-column Diet --o-visualization core-metrics-results/unweighted-unifrac-distance-matrix.qzv

An example of how to output a beta-diversity distance matrix visualization. Also performs PERMANOVA test.

In [ ]:
qiime emperor plot --i-pcoa core-metrics-results/bray_curtis_pcoa_results.qza --m-metadata-file /Users/madibaltagulov/Documents/16s-Madi/TPT_diet-selected/TPT_metadata.txt --o-visualization core-metrics-results/bray-curtis-distance-matrix.qzv

An example of how to output an emperor plot distance matrix (Principal Coordinate Analysis). It is a summation of beta diversity analyses of our data that is plotted on several axes. Percentage labels indicate the amount of variance explained by an axis.

In [ ]:
qiime feature-classifier classify-sklearn --i-classifier /Users/madibaltagulov/Documents/16s-Madi/TPT_diet-selected/16S_reads/2022.10.backbone.full-length.nb.qza --i-reads dada2-rep-seqs.qza --o-classification taxonomy.qza


Creating an actual taxonomy table using our reads and aligning them with Greenhils or Silva classifiers (database of bacteria genomes).

In [ ]:
qiime metadata tabulate --m-input-file taxonomy.qza --o-visualization taxonomy.qzv

Visualizing the taxonomy table

In [ ]:
qiime taxa barplot --i-table table.qza --i-taxonomy taxonomy.qza --m-metadata-file /Users/madibaltagulov/Documents/16s-Madi/TPT_diet-selected/TPT_metadata.txt --o-visualization taxa-bar-plots.qzv

Creating a bar plot that contains our taxonomy table, grouped by metadata categories. The bar plot is customizable by taxa level, treatment, frequence. It also demonstrated relative abundance. In 16S, we look at family or genus level usually.

In [ ]:
qiime taxa collapse --i-table table.qza --i-taxonomy taxonomy.qza --p-level 6 --o-collapsed-table genus-table.qza


Collapsing our taxonomy level to a specific taxonomic level (in this example it's genus). There maybe be 0 values, as well as reads that are not determined down to the genus level.

In [ ]:
qiime composition add-pseudocount --i-table genus-table.qza --o-composition-table genus_comp_table.qza


Adds a pseudocount, transforming 0 values in the table to 1, as well as adding 1 to every value.

In [ ]:
qiime composition ancom --i-table genus_comp_table.qza --m-metadata-file /Users/madibaltagulov/Documents/16s-Madi/TPT_diet-selected/TPT_metadata.txt --m-metadata-column Diet --o-visualization genus-ancom-Diet.qzv


Runs ANCOM on our data. ANCOM is a PERMANOVA-like test, which is gonna test what groups/taxa are significantly different among our diets (3 categories, 1 variable, non-zero values).

In [ ]:
jupyter notebook

Code to open this through terminal.